### Tranform Order data - Explode Arrays
- 1.Access Elements from the JSON object 
- 2.Deduplicate Array Elements 
- 3.Explode Arrays
- 4.write the Transformed Data to silver schema 


In [0]:
df_orders =spark.table("gizmobox.silver.py_orders")
display(df_orders)

1.Access Elements from the JSON object

In [0]:
from pyspark.sql import functions as F

In [0]:
df_acc = (
    df_orders.select(
        "orders.order_id",
        "orders.order_status",
        "orders.payment_method",
        "orders.total_amount",
        "orders.transaction_timestamp",
        "orders.customer_id",
        F.array_distinct("orders.items").alias("items")
    )
)
display(df_acc)

2.Deduplicate Array Elements

In [0]:
df_acc = (
    df_orders.select(
        "orders.order_id",
        "orders.order_status",
        "orders.payment_method",
        "orders.total_amount",
        "orders.transaction_timestamp",
        "orders.customer_id",
        F.array_distinct("orders.items").alias("items")
    )
)
display(df_acc)

3.Explode Arrays

In [0]:
df_explode =(df_acc.select(
        "order_id",
        "order_status",
        "payment_method",
        "total_amount",
        "transaction_timestamp",
        "customer_id",
        F.explode("items").alias("item")

))
display(df_explode)

4.write the Transformed Data to silver schema

In [0]:
df_order_items = (
    df_explode.select(
        "order_id",
        "order_status",
        "payment_method",
        "total_amount",
        "transaction_timestamp",
        "customer_id",
        "item.item_id",
        "item.name",
        "item.price",
        "item.quantity",
        "item.category",
        "item.details.brand",
        "item.details.color"
    )
)
display(df_order_items)

In [0]:
df_order_items.writeTo("gizmobox.silver.py_order_items").createOrReplace()

In [0]:
%sql
select * from gizmobox.silver.py_order_items;